In [5]:
import os, json
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import json_util as bsonju

# --- setup ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
db_name = os.getenv("MONGODB_DATABASE")
col = MongoClient(uri, serverSelectionTimeoutMS=5000)[db_name]["sales_invoices"]

# --- parameter (change this only) ---
receipt_number = "R-000129"

def _created_iso(it):
    v = it.get("created")
    if isinstance(v, dict) and "$date" in v: return v["$date"]
    if isinstance(v, str): return v
    return ""

def _num_item_no(s):
    try: return int(str(s))
    except: return 10**9

# --- find the only empty form ---
docs = list(col.find({"is_empty": True}).limit(2))
if len(docs) != 1:
    print(json.dumps({"ok": False, "reason": "need_exactly_one_empty_form", "count": len(docs)}, indent=2))
else:
    doc = docs[0]
    before_items = list(doc.get("items", []))
    before_len = len(before_items)

    # --- delete item(s) with matching receipt_number ---
    pull_res = col.update_one({"_id": doc["_id"]}, {"$pull": {"items": {"receipt_number": receipt_number}}})

    # Re-read the document
    updated = col.find_one({"_id": doc["_id"]})
    items_after_delete = list(updated.get("items", []))
    removed_count = before_len - len(items_after_delete)

    # --- reorder item_number to "1".."n" deterministically ---
    if items_after_delete:
        items_sorted = sorted(
            items_after_delete,
            key=lambda it: (_created_iso(it) == "", _created_iso(it), _num_item_no(it.get("item_number", "0")))
        )
        for i, it in enumerate(items_sorted, start=1):
            it["item_number"] = str(i)
        set_res = col.update_one({"_id": doc["_id"]}, {"$set": {"items": items_sorted}})
        final_doc = col.find_one({"_id": doc["_id"]}, {"items": 1, "id": 1, "current_form_qr_code": 1, "is_empty": 1})
        final_numbers = [it.get("item_number") for it in final_doc.get("items", [])]
    else:
        # nothing left; skip renumber (is_empty remains True)
        set_res = None
        final_doc = {"items": [], "id": updated.get("id"), "current_form_qr_code": updated.get("current_form_qr_code"), "is_empty": updated.get("is_empty")}
        final_numbers = []

    # --- output ---
    print(bsonju.dumps({
        "ok": True,
        "message": "Deleted by receipt_number, then renumbered remaining items (is_empty unchanged).",
        "receipt_number": receipt_number,
        "removed_count": max(removed_count, 0),
        "matched_doc": pull_res.matched_count,
        "modified_doc_pull": pull_res.modified_count,
        "modified_doc_set": (set_res.modified_count if set_res else 0),
        "document_meta": {
            "id": final_doc.get("id"),
            "current_form_qr_code": final_doc.get("current_form_qr_code"),
            "is_empty": final_doc.get("is_empty"),
            "items_length": len(final_numbers)
        },
        "item_numbers_after": final_numbers
    }, indent=2))


{
  "ok": true,
  "message": "Deleted by receipt_number, then renumbered remaining items (is_empty unchanged).",
  "receipt_number": "R-000129",
  "removed_count": 1,
  "matched_doc": 1,
  "modified_doc_pull": 1,
  "modified_doc_set": 1,
  "document_meta": {
    "id": "6b4a1918-5d36-44b6-be53-1d2cbba5b64b",
    "current_form_qr_code": "20251014",
    "is_empty": true,
    "items_length": 6
  },
  "item_numbers_after": [
    "1",
    "2",
    "3",
    "4",
    "5",
    "6"
  ]
}
